In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import sys
sys.path.append("../scripts/")
import benchmarking

In [20]:
path = '../outputs/'
ranking_files = {
    'ablation': 'ablation/task1_latent.csv',
    'de': 'differential_expression/norman_DE_control.csv',
    'eg_abssum': 'expected_gradients/task_1_absolute_expected_grads.csv',
#     'ig_abssum': 'integrated_gradients/task_1_absolute_sum_integrated_grads.csv',
    'mean': 'baselines/task1_mean.csv',
    'random': 'baselines/task1_random.csv',
}
ranking_dfs = {explainer: pd.read_csv(path + file_name, index_col=0) for explainer, file_name in ranking_files.items()}

Invert DE because p-values go in the wrong direction.

In [21]:
ranking_dfs['de'] = -ranking_dfs['de']

In [22]:
auc_scores = dict()
for explainer, df in ranking_dfs.items():
    print(explainer)
    auc_scores[explainer] = benchmarking.roc_auc_crispr(df)

ablation
de
eg_abssum
mean
random


In [23]:
auc_scores

{'ablation': 0.6588555441106874,
 'de': 0.9705116784053897,
 'eg_abssum': 0.8141745208363622,
 'mean': 0.8689830667503895,
 'random': 0.4559094185078985}

Note that `mean` only works here because we've trained on Norman19, a CRISPRa dataset. It would definitely not be the case for a CRISPRi/ko screen.